- 1. 多输入通道

In [2]:
import torch
from d2l import torch as d2l

# 对每个通道输入的二维张量和卷积核的二维张量进行互相关运算，再对通道求和
def corr2d_multi_in(X, K):
    # 遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

X = torch.tensor([
    [[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
    [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]
])
K = torch.tensor([
    [[0.0, 1.0], [2.0, 3.0]],
    [[1.0, 2.0], [3.0, 4.0]]
])

# print(X.shape)
# print(K.shape)
print(corr2d_multi_in(X, K))

tensor([[ 56.,  72.],
        [104., 120.]])


- 2. 多输出通道

In [3]:
def corr2d_multi_in_out(X, K):
    # print([corr2d_multi_in(X, k) for k in K])
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack([K, K+1, K+2], 0)
print(K.shape) # [3,2,2,2]

print(corr2d_multi_in_out(X, K))

torch.Size([3, 2, 2, 2])
tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])


- 3.  1 * 1 卷积层

使用全连接层实现 1x1 卷积

In [7]:
import torch
from d2l import torch as d2l

# def corr2d_multi_in(X, K):
#     # 遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
#     return sum(d2l.corr2d(x, k) for x, k in zip(X, K))
#
# def corr2d_multi_in_out(X, K):
#     # print([corr2d_multi_in(X, k) for k in K])
#     return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape # c_i:输入通道数, h:输入高度, w:输入宽度
    c_o = K.shape[0] # c_o: 卷积层通道数
    X = X.reshape(c_i, h * w) # 将高、宽展平
    K = K.reshape(c_o, c_i)
    # print(K)
    # 全链路层的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape(c_o, h, w) # 保持 二维矩阵结构一致

X = torch.normal(0, 1, (3, 3, 3)) # 3个通道, 每个通道的 高、宽 分别是 3, 3
K = torch.normal(0, 1, (2, 3, 1, 1))
# print('X', X)
print('K', K)

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
print('Y1', Y1, 'Y2', Y2)

K tensor([[[[ 1.1257]],

         [[ 0.4902]],

         [[-0.8148]]],


        [[[-1.5501]],

         [[-0.9508]],

         [[-1.0045]]]])
Y1 tensor([[[ 2.3831,  1.8367,  1.6773],
         [ 0.4058,  0.6301,  0.4983],
         [ 0.2635,  0.1777, -0.9283]],

        [[-4.9811, -5.1364, -1.2088],
         [ 0.4631, -0.2867, -1.0314],
         [ 1.2518, -0.9913, -0.2379]]]) Y2 tensor([[[ 2.3831,  1.8367,  1.6773],
         [ 0.4058,  0.6301,  0.4983],
         [ 0.2635,  0.1777, -0.9283]],

        [[-4.9811, -5.1364, -1.2088],
         [ 0.4631, -0.2867, -1.0314],
         [ 1.2518, -0.9913, -0.2379]]])
